# 第二课 词向量

褚则伟 zeweichu@gmail.com

第二课学习目标
- 学习词向量的概念
- 用Skip-thought模型训练词向量
- 学习使用PyTorch dataset和dataloader
- 学习定义PyTorch模型
- 学习torch.nn中常见的Module
    - Embedding
- 学习常见的PyTorch operations
    - bmm
    - logsigmoid
- 保存和读取PyTorch模型
    

第二课使用的训练数据可以从以下链接下载到。

链接:https://pan.baidu.com/s/1tFeK3mXuVXEy3EMarfeWvg  密码:v2z5

在这一份notebook中，我们会（尽可能）尝试复现论文[Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)中训练词向量的方法. 我们会实现Skip-gram模型，并且使用论文中noice contrastive sampling的目标函数。

这篇论文有很多模型实现的细节，这些细节对于词向量的好坏至关重要。我们虽然无法完全复现论文中的实验结果，主要是由于计算资源等各种细节原因，但是我们还是可以大致展示如何训练词向量。

以下是一些我们没有实现的细节
- subsampling：参考论文section 2.3

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from torch.nn.parameter import Parameter

from collections import Counter
import numpy as np
import random
import math

import pandas as pd
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

USE_CUDA = torch.cuda.is_available()

# 为了保证实验结果可以复现，我们经常会把各种random seed固定在某一个值
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)
    
# 设定一些超参数
    
K = 10 # number of negative samples
C = 2 # nearby words threshold
NUM_EPOCHS = 2 # The number of epochs of training
MAX_VOCAB_SIZE = 9000 # the vocabulary size
BATCH_SIZE = 32 # the batch size
LEARNING_RATE = 0.2 # the initial learning rate
EMBEDDING_SIZE = 25
       
    
LOG_FILE = "word-embedding.log"

# tokenize函数，把一篇文本转化成一个个单词
def word_tokenize(text):
    return text.split()

- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [2]:
with open("text8.train.txt", "r") as fin:
    text = fin.read()
    
text = [w for w in word_tokenize(text.lower())]
vocab = dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))
vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))
idx_to_word = [word for word in vocab.keys()] 
word_to_idx = {word:i for i, word in enumerate(idx_to_word)}

word_counts = np.array([count for count in vocab.values()], dtype=np.float32)
word_freqs = word_counts / np.sum(word_counts)
word_freqs = word_freqs ** (3./4.)
word_freqs = word_freqs / np.sum(word_freqs) # 用来做 negative sampling
VOCAB_SIZE = len(idx_to_word)
VOCAB_SIZE

9000

### 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字，然后用subsampling预处理这些文字。
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

In [3]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        ''' text: a list of words, all text from the training dataset
            word_to_idx: the dictionary from word to idx
            idx_to_word: idx to word mapping
            word_freq: the frequency of each word
            word_counts: the word counts
        '''
        super(WordEmbeddingDataset, self).__init__()
        self.text_encoded = [word_to_idx.get(t, VOCAB_SIZE-1) for t in text]
        self.text_encoded = torch.Tensor(self.text_encoded).long()
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.word_freqs = torch.Tensor(word_freqs)
        self.word_counts = torch.Tensor(word_counts)
        
    def __len__(self):
        ''' 返回整个数据集（所有单词）的长度
        '''
        return len(self.text_encoded)
        
    def __getitem__(self, idx):
        ''' 这个function返回以下数据用于训练
            - 中心词
            - 这个单词附近的(positive)单词
            - 随机采样的K个单词作为negative sample
        '''
        center_word = self.text_encoded[idx]
        pos_indices = list(range(idx-C, idx)) + list(range(idx+1, idx+C+1))
        pos_indices = [i%len(self.text_encoded) for i in pos_indices]
        pos_words = self.text_encoded[pos_indices] 
        neg_words = torch.multinomial(self.word_freqs, K * pos_words.shape[0], True)
        
        return center_word, pos_words, neg_words 

创建dataset和dataloader

In [4]:
dataset = WordEmbeddingDataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)
dataloader = tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)     

In [5]:
next(iter(dataloader))

[tensor([  48, 1495, 1799, 2312, 2329, 4869,    8,    0, 8746,   71,  772,    7,
         2837,    0, 8999, 1746, 7122,    6,   10, 1956, 8999,   33,   17,  230,
         8999,    5,  291,  640, 8999,  103, 2505,  540]),
 tensor([[   0,  868,  363,   13],
         [8999, 3751, 5168,  932],
         [   3,    5,  345,    1],
         [8075,    2, 1008,    2],
         [8999, 3981, 1436,  582],
         [   4,    5,  213,   27],
         [1761,    3,   22,    8],
         [2813,   19, 8999, 2077],
         [  44, 2189, 5529,   26],
         [   0, 2266, 4809, 2876],
         [4219, 1173, 1262, 8999],
         [   9,    7,    7,  599],
         [   5, 2252, 8999,    0],
         [  70,   38, 4720,    1],
         [4719,    2,   39, 2117],
         [ 254,  387,   23,    0],
         [  54,   11, 6962, 4473],
         [  10, 7479, 5115, 8999],
         [  32,  557,  657,    5],
         [ 216,    6, 1051, 4695],
         [1463,    6,    0, 8999],
         [8999,    1,    0, 8999],
         

### 定义PyTorch模型

In [6]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        ''' 初始化输出和输出embedding
        '''
        super(EmbeddingModel, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        
        initrange = 0.5 / self.embed_size
        self.out_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        self.out_embed.weight.data.uniform_(-initrange, initrange)
        
        
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        self.in_embed.weight.data.uniform_(-initrange, initrange)
        
        
    def forward(self, input_labels, pos_labels, neg_labels):
        '''
        input_labels: 中心词, [batch_size]
        pos_labels: 中心词周围 context window 出现过的单词 [batch_size * (window_size * 2)]
        neg_labelss: 中心词周围没有出现过的单词，从 negative sampling 得到 [batch_size, (window_size * 2 * K)]
        
        return: loss, [batch_size]
        '''
        
        batch_size = input_labels.size(0)
        
        input_embedding = self.in_embed(input_labels) # B * embed_size
        pos_embedding = self.out_embed(pos_labels) # B * (2*C) * embed_size
        neg_embedding = self.out_embed(neg_labels) # B * (2*C * K) * embed_size
      
        log_pos = torch.bmm(pos_embedding, input_embedding.unsqueeze(2)).squeeze() # B * (2*C)
        log_neg = torch.bmm(neg_embedding, -input_embedding.unsqueeze(2)).squeeze() # B * (2*C*K)

        log_pos = F.logsigmoid(log_pos).sum(1)
        log_neg = F.logsigmoid(log_neg).sum(1) # batch_size
       
        loss = log_pos + log_neg
        
        return -loss
    
    def input_embeddings(self):
        return self.in_embed.weight.data.cpu().numpy()
        

定义一个模型以及把模型移动到GPU

In [7]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()

下面是评估模型的代码，以及训练模型的代码

In [8]:
def evaluate(filename, embedding_weights): 
    if filename.endswith(".csv"):
        data = pd.read_csv(filename, sep=",")
    else:
        data = pd.read_csv(filename, sep="\t")
    human_similarity = []
    model_similarity = []
    for i in data.iloc[:, 0:2].index:
        word1, word2 = data.iloc[i, 0], data.iloc[i, 1]
        if word1 not in word_to_idx or word2 not in word_to_idx:
            continue
        else:
            word1_idx, word2_idx = word_to_idx[word1], word_to_idx[word2]
            word1_embed, word2_embed = embedding_weights[[word1_idx]], embedding_weights[[word2_idx]]
            model_similarity.append(float(sklearn.metrics.pairwise.cosine_similarity(word1_embed, word2_embed)))
            human_similarity.append(float(data.iloc[i, 2]))

    return scipy.stats.spearmanr(human_similarity, model_similarity)# , model_similarity

def find_nearest(word):
    index = word_to_idx[word]
    embedding = embedding_weights[index]
    cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]]

训练模型：
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，以及在验证数据集上做模型的评估

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
for e in range(NUM_EPOCHS):
    for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):
        
        
        # TODO
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()
            
        optimizer.zero_grad()
        loss = model(input_labels, pos_labels, neg_labels).mean()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
#            with open(LOG_FILE, "a") as fout:
#                fout.write("epoch: {}, iter: {}, loss: {}\n".format(e, i, loss.item()))
                print("epoch: {}, iter: {}, loss: {}".format(e, i, loss.item()))
            
        
        if i % 2000 == 0:
            embedding_weights = model.input_embeddings()
            sim_simlex = evaluate(r".\embedding\simlex-999.txt", embedding_weights)
            sim_men = evaluate(r".\embedding\\men.txt", embedding_weights)
            sim_353 = evaluate(r".\embedding\wordsim353.csv", embedding_weights)
#            with open(LOG_FILE, "a") as fout:
            print("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
#                fout.write("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
#                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                
    embedding_weights = model.input_embeddings()
    np.save("embedding-{}".format(EMBEDDING_SIZE), embedding_weights)
    torch.save(model.state_dict(), "embedding-{}.th".format(EMBEDDING_SIZE))

epoch: 0, iter: 0, loss: 30.498878479003906
epoch: 0, iteration: 0, simlex-999: SpearmanrResult(correlation=0.038419267682371507, pvalue=0.3487076419792051), men: SpearmanrResult(correlation=0.03038333081320989, pvalue=0.2738394063825371), sim353: SpearmanrResult(correlation=-0.02851728381964284, pvalue=0.6575798953240686), nearest to monster: ['monster', 'drawn', 'affects', 'focused', 'guidance', 'push', 'compared', 'bismarck', 'encountered', 'buddhist']

epoch: 0, iter: 100, loss: 30.487579345703125
epoch: 0, iter: 200, loss: 30.173709869384766
epoch: 0, iter: 300, loss: 29.176044464111328
epoch: 0, iter: 400, loss: 28.22866439819336
epoch: 0, iter: 500, loss: 27.254440307617188
epoch: 0, iter: 600, loss: 25.54012680053711
epoch: 0, iter: 700, loss: 23.82415008544922
epoch: 0, iter: 800, loss: 24.10273551940918
epoch: 0, iter: 900, loss: 24.398141860961914
epoch: 0, iter: 1000, loss: 25.676937103271484
epoch: 0, iter: 1100, loss: 21.91301918029785
epoch: 0, iter: 1200, loss: 20.88068

epoch: 0, iter: 12100, loss: 13.25427532196045
epoch: 0, iter: 12200, loss: 12.95503044128418
epoch: 0, iter: 12300, loss: 13.252090454101562
epoch: 0, iter: 12400, loss: 13.105239868164062
epoch: 0, iter: 12500, loss: 13.69926643371582
epoch: 0, iter: 12600, loss: 13.682611465454102
epoch: 0, iter: 12700, loss: 13.328731536865234
epoch: 0, iter: 12800, loss: 13.191139221191406
epoch: 0, iter: 12900, loss: 12.566657066345215
epoch: 0, iter: 13000, loss: 13.497055053710938
epoch: 0, iter: 13100, loss: 13.831032752990723
epoch: 0, iter: 13200, loss: 12.346025466918945
epoch: 0, iter: 13300, loss: 14.12915325164795
epoch: 0, iter: 13400, loss: 13.068685531616211
epoch: 0, iter: 13500, loss: 13.708019256591797
epoch: 0, iter: 13600, loss: 13.4109468460083
epoch: 0, iter: 13700, loss: 12.937657356262207
epoch: 0, iter: 13800, loss: 13.079425811767578
epoch: 0, iter: 13900, loss: 13.839799880981445
epoch: 0, iter: 14000, loss: 13.190155982971191
epoch: 0, iteration: 14000, simlex-999: Spearm

epoch: 0, iter: 24100, loss: 12.252010345458984
epoch: 0, iter: 24200, loss: 11.623882293701172
epoch: 0, iter: 24300, loss: 12.626989364624023
epoch: 0, iter: 24400, loss: 12.15478515625
epoch: 0, iter: 24500, loss: 12.346567153930664
epoch: 0, iter: 24600, loss: 11.919984817504883
epoch: 0, iter: 24700, loss: 12.55777359008789
epoch: 0, iter: 24800, loss: 12.016989707946777
epoch: 0, iter: 24900, loss: 12.572108268737793
epoch: 0, iter: 25000, loss: 11.995954513549805
epoch: 0, iter: 25100, loss: 12.41183853149414
epoch: 0, iter: 25200, loss: 11.38509750366211
epoch: 0, iter: 25300, loss: 12.902754783630371
epoch: 0, iter: 25400, loss: 11.913442611694336
epoch: 0, iter: 25500, loss: 12.687858581542969
epoch: 0, iter: 25600, loss: 11.980302810668945
epoch: 0, iter: 25700, loss: 12.748454093933105
epoch: 0, iter: 25800, loss: 12.460587501525879
epoch: 0, iter: 25900, loss: 12.35887336730957
epoch: 0, iter: 26000, loss: 12.11594295501709
epoch: 0, iteration: 26000, simlex-999: Spearmanr

epoch: 0, iter: 36100, loss: 11.610977172851562
epoch: 0, iter: 36200, loss: 12.651154518127441
epoch: 0, iter: 36300, loss: 12.242701530456543
epoch: 0, iter: 36400, loss: 12.130355834960938
epoch: 0, iter: 36500, loss: 11.517946243286133
epoch: 0, iter: 36600, loss: 12.078970909118652
epoch: 0, iter: 36700, loss: 12.557184219360352
epoch: 0, iter: 36800, loss: 11.657432556152344
epoch: 0, iter: 36900, loss: 12.103632926940918
epoch: 0, iter: 37000, loss: 12.138496398925781
epoch: 0, iter: 37100, loss: 11.97175407409668
epoch: 0, iter: 37200, loss: 12.5131196975708
epoch: 0, iter: 37300, loss: 12.814860343933105
epoch: 0, iter: 37400, loss: 11.409767150878906
epoch: 0, iter: 37500, loss: 12.247803688049316
epoch: 0, iter: 37600, loss: 12.004988670349121
epoch: 0, iter: 37700, loss: 11.94013786315918
epoch: 0, iter: 37800, loss: 11.871380805969238
epoch: 0, iter: 37900, loss: 11.786941528320312
epoch: 0, iter: 38000, loss: 13.039443969726562
epoch: 0, iteration: 38000, simlex-999: Spea

epoch: 0, iter: 48100, loss: 11.51461410522461
epoch: 0, iter: 48200, loss: 11.825233459472656
epoch: 0, iter: 48300, loss: 11.835733413696289
epoch: 0, iter: 48400, loss: 11.552790641784668
epoch: 0, iter: 48500, loss: 11.608691215515137
epoch: 0, iter: 48600, loss: 12.621800422668457
epoch: 0, iter: 48700, loss: 11.954909324645996
epoch: 0, iter: 48800, loss: 11.846529006958008
epoch: 0, iter: 48900, loss: 11.820432662963867
epoch: 0, iter: 49000, loss: 11.999134063720703
epoch: 0, iter: 49100, loss: 11.94824504852295
epoch: 0, iter: 49200, loss: 11.539299964904785
epoch: 0, iter: 49300, loss: 11.879488945007324
epoch: 0, iter: 49400, loss: 12.390554428100586
epoch: 0, iter: 49500, loss: 11.956644058227539
epoch: 0, iter: 49600, loss: 11.819853782653809
epoch: 0, iter: 49700, loss: 12.099848747253418
epoch: 0, iter: 49800, loss: 11.637537002563477
epoch: 0, iter: 49900, loss: 11.650900840759277
epoch: 0, iter: 50000, loss: 12.219585418701172
epoch: 0, iteration: 50000, simlex-999: Sp

...

epoch: 1, iter: 422100, loss: 10.962005615234375
epoch: 1, iter: 422200, loss: 11.717196464538574
epoch: 1, iter: 422300, loss: 11.740622520446777
epoch: 1, iter: 422400, loss: 11.439126968383789
epoch: 1, iter: 422500, loss: 11.251435279846191
epoch: 1, iter: 422600, loss: 11.98227596282959
epoch: 1, iter: 422700, loss: 11.151405334472656
epoch: 1, iter: 422800, loss: 11.325945854187012
epoch: 1, iter: 422900, loss: 10.862709999084473
epoch: 1, iter: 423000, loss: 11.037946701049805
epoch: 1, iter: 423100, loss: 11.99278450012207
epoch: 1, iter: 423200, loss: 10.962024688720703
epoch: 1, iter: 423300, loss: 10.907966613769531
epoch: 1, iter: 423400, loss: 10.773236274719238
epoch: 1, iter: 423500, loss: 11.348245620727539
epoch: 1, iter: 423600, loss: 11.209400177001953
epoch: 1, iter: 423700, loss: 10.71605396270752
epoch: 1, iter: 423800, loss: 10.275297164916992
epoch: 1, iter: 423900, loss: 11.691515922546387
epoch: 1, iter: 424000, loss: 11.235740661621094
epoch: 1, iteration: 42

epoch: 1, iter: 434100, loss: 11.398099899291992
epoch: 1, iter: 434200, loss: 11.275341987609863
epoch: 1, iter: 434300, loss: 11.040188789367676
epoch: 1, iter: 434400, loss: 11.44877815246582
epoch: 1, iter: 434500, loss: 11.126958847045898
epoch: 1, iter: 434600, loss: 11.022445678710938
epoch: 1, iter: 434700, loss: 10.526141166687012
epoch: 1, iter: 434800, loss: 10.883358001708984
epoch: 1, iter: 434900, loss: 11.001891136169434
epoch: 1, iter: 435000, loss: 10.739945411682129
epoch: 1, iter: 435100, loss: 12.499483108520508
epoch: 1, iter: 435200, loss: 12.20584774017334
epoch: 1, iter: 435300, loss: 11.220914840698242
epoch: 1, iter: 435400, loss: 11.303383827209473
epoch: 1, iter: 435500, loss: 10.84464168548584
epoch: 1, iter: 435600, loss: 11.762062072753906
epoch: 1, iter: 435700, loss: 12.285225868225098
epoch: 1, iter: 435800, loss: 11.432060241699219
epoch: 1, iter: 435900, loss: 11.49901294708252
epoch: 1, iter: 436000, loss: 10.682817459106445
epoch: 1, iteration: 436

epoch: 1, iter: 446100, loss: 10.895466804504395
epoch: 1, iter: 446200, loss: 11.517123222351074
epoch: 1, iter: 446300, loss: 11.391008377075195
epoch: 1, iter: 446400, loss: 11.51754093170166
epoch: 1, iter: 446500, loss: 11.363222122192383
epoch: 1, iter: 446600, loss: 11.756290435791016
epoch: 1, iter: 446700, loss: 11.249637603759766
epoch: 1, iter: 446800, loss: 10.764711380004883
epoch: 1, iter: 446900, loss: 11.212006568908691
epoch: 1, iter: 447000, loss: 12.00946044921875
epoch: 1, iter: 447100, loss: 11.911036491394043
epoch: 1, iter: 447200, loss: 11.14554214477539
epoch: 1, iter: 447300, loss: 11.05186939239502
epoch: 1, iter: 447400, loss: 11.011931419372559
epoch: 1, iter: 447500, loss: 11.680644035339355
epoch: 1, iter: 447600, loss: 11.899770736694336
epoch: 1, iter: 447700, loss: 10.516495704650879
epoch: 1, iter: 447800, loss: 10.518952369689941
epoch: 1, iter: 447900, loss: 11.454275131225586
epoch: 1, iter: 448000, loss: 10.575553894042969
epoch: 1, iteration: 448

epoch: 1, iter: 458100, loss: 11.59527587890625
epoch: 1, iter: 458200, loss: 11.086811065673828
epoch: 1, iter: 458300, loss: 10.718034744262695
epoch: 1, iter: 458400, loss: 11.232890129089355
epoch: 1, iter: 458500, loss: 11.93146800994873
epoch: 1, iter: 458600, loss: 10.830869674682617
epoch: 1, iter: 458700, loss: 11.208268165588379
epoch: 1, iter: 458800, loss: 10.195466995239258
epoch: 1, iter: 458900, loss: 11.265641212463379
epoch: 1, iter: 459000, loss: 11.847845077514648
epoch: 1, iter: 459100, loss: 11.914114952087402
epoch: 1, iter: 459200, loss: 11.146728515625
epoch: 1, iter: 459300, loss: 10.803190231323242
epoch: 1, iter: 459400, loss: 11.36327838897705
epoch: 1, iter: 459500, loss: 11.565722465515137
epoch: 1, iter: 459600, loss: 11.303350448608398
epoch: 1, iter: 459700, loss: 11.040358543395996
epoch: 1, iter: 459800, loss: 11.079276084899902
epoch: 1, iter: 459900, loss: 11.18036937713623
epoch: 1, iter: 460000, loss: 11.595579147338867
epoch: 1, iteration: 460000

epoch: 1, iter: 470100, loss: 11.579556465148926
epoch: 1, iter: 470200, loss: 11.333196640014648
epoch: 1, iter: 470300, loss: 11.586036682128906
epoch: 1, iter: 470400, loss: 11.622262954711914
epoch: 1, iter: 470500, loss: 11.52660846710205
epoch: 1, iter: 470600, loss: 11.743627548217773
epoch: 1, iter: 470700, loss: 11.7948579788208
epoch: 1, iter: 470800, loss: 10.743227005004883
epoch: 1, iter: 470900, loss: 11.618430137634277
epoch: 1, iter: 471000, loss: 10.725007057189941
epoch: 1, iter: 471100, loss: 11.477409362792969
epoch: 1, iter: 471200, loss: 11.633712768554688
epoch: 1, iter: 471300, loss: 11.383800506591797
epoch: 1, iter: 471400, loss: 11.573264122009277
epoch: 1, iter: 471500, loss: 11.79548454284668
epoch: 1, iter: 471600, loss: 10.598237037658691
epoch: 1, iter: 471700, loss: 10.672428131103516
epoch: 1, iter: 471800, loss: 10.925034523010254
epoch: 1, iter: 471900, loss: 12.021942138671875
epoch: 1, iter: 472000, loss: 10.825090408325195
epoch: 1, iteration: 472

In [17]:
model.load_state_dict(torch.load("embedding-{}.th".format(EMBEDDING_SIZE)))

<All keys matched successfully>

## 在 MEN 和 Simplex-999 数据集上做评估

In [19]:
embedding_weights = model.input_embeddings()
print("simlex-999", evaluate(r".\embedding\simlex-999.txt", embedding_weights))
print("men", evaluate(r".\embedding\men.txt", embedding_weights))
print("wordsim353", evaluate(r".\embedding\wordsim353.csv", embedding_weights))

simlex-999 SpearmanrResult(correlation=0.17346411394508127, pvalue=2.0277156165168663e-05)
men SpearmanrResult(correlation=0.45392756043920385, pvalue=5.045440536459275e-67)
wordsim353 SpearmanrResult(correlation=0.42494185324583417, pvalue=4.055988368438879e-12)


## 寻找nearest neighbors

In [20]:
for word in ["good", "fresh", "monster", "green", "like", "america", "chicago", "work", "computer", "language"]:
    print(word, find_nearest(word))

good ['good', 'bad', 'better', 'magical', 'mere', 'conscious', 'worse', 'unique', 'perfect', 'complete']
fresh ['fresh', 'wet', 'milk', 'grain', 'warm', 'shallow', 'mild', 'alcohol', 'breathing', 'kidney']
monster ['monster', 'cow', 'bull', 'snake', 'whale', 'hammer', 'tiger', 'flower', 'bat', 'skull']
green ['green', 'blue', 'yellow', 'purple', 'orange', 'pink', 'white', 'red', 'shaped', 'black']
like ['like', 'resembling', 'bearing', 'etc', 'resemble', 'exhibit', 'bear', 'similarly', 'eating', 'wherein']
america ['america', 'australia', 'africa', 'england', 'korea', 'europe', 'berlin', 'carolina', 'mexico', 'wales']
chicago ['chicago', 'brooklyn', 'boston', 'toronto', 'buffalo', 'cincinnati', 'seattle', 'detroit', 'pittsburgh', 'philadelphia']
work ['work', 'writing', 'comment', 'account', 'vision', 'talent', 'reading', 'criticism', 'dialogue', 'appearance']
computer ['computer', 'animation', 'editing', 'interactive', 'digital', 'micro', 'gaming', 'unix', 'software', 'graphics']
lang

## 单词之间的关系

In [21]:
man_idx = word_to_idx["man"] 
king_idx = word_to_idx["king"] 
woman_idx = word_to_idx["woman"]
embedding = embedding_weights[woman_idx] - embedding_weights[man_idx] + embedding_weights[king_idx]
cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
for i in cos_dis.argsort()[:20]:
    print(idx_to_word[i])

king
viii
tsar
bonaparte
augustus
constantine
vii
alfonso
baldwin
julius
afonso
empress
clement
byron
son
emperor
xii
charlemagne
frederick
pope
